<a href="https://colab.research.google.com/github/SarahBarbosa/AluraDataChallenge_2/blob/main/01_Pipeline_de_extracao_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pipeline de extração de dados


## Importação das bibliotecas

In [ ]:
%%capture
try:
    import py7zr
    from ftplib import FTP
except ModuleNotFoundError:
    !pip install py7zr
    !pip install fastparquet
    import py7zr
    from ftplib import FTP

import os
import numpy as np
import pandas as pd

import sys
sys.path.append('/content/drive/MyDrive/Projeto - RAIS')

from google.colab import drive
drive.mount('/content/drive')

## Conexão e download dos arquivos compactados

In [ ]:
from download_arquivos_rais import download_arquivos

In [ ]:
base = '/content/drive/MyDrive/Projeto - RAIS/base/'

lista_de_estados = ['RAIS_VINC_PUB_CENTRO_OESTE',
                    'RAIS_VINC_PUB_MG_ES_RJ',
                    'RAIS_VINC_PUB_NORDESTE',
                    'RAIS_VINC_PUB_NORTE',
                    'RAIS_VINC_PUB_SP',
                    'RAIS_VINC_PUB_SUL']

ftp = FTP('ftp.mtps.gov.br')
download_arquivos(2020, base, ftp, lista_de_estados)

Download concluído (1 / 6): RAIS_VINC_PUB_CENTRO_OESTE.7z
Download concluído (2 / 6): RAIS_VINC_PUB_MG_ES_RJ.7z
Download concluído (3 / 6): RAIS_VINC_PUB_NORDESTE.7z
Download concluído (4 / 6): RAIS_VINC_PUB_NORTE.7z
Download concluído (5 / 6): RAIS_VINC_PUB_SP.7z
Download concluído (6 / 6): RAIS_VINC_PUB_SUL.7z
## Todos os downloads foram concluídos com sucesso.


In [ ]:
ftp = FTP('ftp.mtps.gov.br')
download_arquivos(2021, base, ftp, lista_de_estados)

Download concluído (1 / 6): RAIS_VINC_PUB_CENTRO_OESTE.7z
Download concluído (2 / 6): RAIS_VINC_PUB_MG_ES_RJ.7z
Download concluído (3 / 6): RAIS_VINC_PUB_NORDESTE.7z
Download concluído (4 / 6): RAIS_VINC_PUB_NORTE.7z
Download concluído (5 / 6): RAIS_VINC_PUB_SP.7z
Download concluído (6 / 6): RAIS_VINC_PUB_SUL.7z
## Todos os downloads foram concluídos com sucesso.


## Visualizando o layout dos dados

Infelizmente, ao tentarmos acessar a pasta "vínculos" em `Layout` para visualizar a organização dos dados (nomes das colunas e suas descrições), nos deparamos com um erro. Detectamos um problema relacionado aos caracteres no formato UTF-8. Para superar essa dificuldade, optamos por acessar diretamente esse diretório e baixar o arquivo de lá. A partir desse momento, conseguimos visualizar tanto os nomes quanto as descrições das variáveis em nosso conjunto de dados:

In [ ]:
layout_variaveis = pd.read_csv('/content/drive/MyDrive/Projeto - RAIS/layout.csv')
layout_variaveis.head()

,Nome,Descricao da Variável,Categorias,Valor na Fonte
0,BAIRRO_SP,Bairros do Municipio de São Paulo,(ver planilha correspondente contida neste arq...,NaN
1,BAIRRO FORT,Bairros do município de Fortaleza,(ver planilha correspondente contida neste arq...,NaN
2,BAIRRO_RJ,Bairros do município do Rio de Janeiro,(ver planilha correspondente contida neste arq...,NaN
3,CAUS AFAST 1,Causa do primeiro afastamento do empregado/ser...,ACI TRB TIP,10
4,NaN,NaN,ACI TRB TJT,20


In [ ]:
layout_motivo_desligamento = pd.read_csv('/content/drive/MyDrive/Projeto - RAIS/layout_desligamento.csv')
layout_motivo_desligamento.head()

,Valor do Motivo Desligamento
0,10 - Rescisão com justa causa por iniciativa d...
1,11 - Rescisão sem justa causa por iniciativa d...
2,12 - Término do contrato de trabalho.
3,20 - Rescisão com justa causa por iniciativa d...
4,21 - Rescisão sem justa causa por iniciativa d...


## Download das amostras

Com base nas informações fornecidas nas descrições das colunas no arquivo `layout.csv`, podemos iniciar o processo descartando algumas colunas. Isso nos permitirá reduzir a quantidade de dados aparentemente "desnecessários" que seriam carregados durante o download das amostras. É importante mencionar que estamos trabalhando com uma porção do conjunto completo de dados devido à sua extensão, pois lidar com a totalidade dos dados poderia sobrecarregar o kernel do Colab e nossos códigos.

In [ ]:
colunas_a_excluir = ['Bairros SP',
                     'Bairros Fortaleza',
                     'Bairros RJ',
                     'CNAE 95 Classe',
                     'Distritos SP',
                     'Faixa Etária',
                     'Faixa Hora Contrat',
                     'Ind CEI Vinculado',
                     'Ind Simples',
                     'Mun Trab',
                     'Nacionalidade',
                     'Regiões Adm DF',
                     'Tipo Estab.1',
                     'Vl Remun Dezembro Nom',
                     'Vl Remun Média Nom',
                     'Vl Remun Dezembro (SM)',
                     'Vl Rem Janeiro SC',
                     'Vl Rem Fevereiro SC',
                     'Vl Rem Março SC',
                     'Vl Rem Abril SC',
                     'Vl Rem Maio SC',
                     'Vl Rem Junho SC',
                     'Vl Rem Julho SC',
                     'Vl Rem Agosto SC',
                     'Vl Rem Setembro SC',
                     'Vl Rem Outubro SC',
                     'Vl Rem Novembro SC',
                     'Ano Chegada Brasil']

Vamos importar nossa classe para extrair, amostrar e exportar como dataframe do tipo parquet:

In [ ]:
from processador_de_dados_rais import ProcessadorDeDadosRais

Importante: a função `processar_dados_e_salvar` da `ProcessadorDeDadosRais` espera um parâmetro de indíce, que é justamente o indice do arquivo 7z que será extraíndo, por exemplo, em:



In [ ]:
os.listdir(os.path.join(base, str(2020)))

['RAIS_VINC_PUB_CENTRO_OESTE.7z',
 'RAIS_VINC_PUB_MG_ES_RJ.7z',
 'RAIS_VINC_PUB_NORDESTE.7z',
 'RAIS_VINC_PUB_NORTE.7z',
 'RAIS_VINC_PUB_SP.7z',
 'RAIS_VINC_PUB_SUL.7z']

É importante destacar que ocasionalmente o Kernel pode encerrar. Portanto, se isso ocorrer, por favor, proceda com a reinicialização do ambiente Colab e dos trechos de código a seguir ou coloque `baixa_memoria = True` para melhorar o desempenho (menos consumo de RAM do sistema, em compensação, demorará mais).

In [ ]:
ano = 2020
base = '/content/drive/MyDrive/Projeto - RAIS/base/'
saida = '/content/drive/MyDrive/Projeto - RAIS/extract/'

processador = ProcessadorDeDadosRais(proporcao_amostra = 0.01, semente_aleatoria = 42)
processador.processar_dados_e_salvar(ano, 0, base, saida, colunas_a_excluir, baixa_memoria = False)

Acessando o diretório: RAIS_VINC_PUB_CENTRO_OESTE.7z 

Extraindo arquivo: RAIS_VINC_PUB_CENTRO_OESTE.txt
O arquivo já foi extraído anteriormente: /content/drive/MyDrive/Projeto - RAIS/extract/2020/RAIS_VINC_PUB_CENTRO_OESTE.txt

Criando DataFrame para: CENTRO_OESTE


'O dataframe em formato parquet já foi salvo anteriormente.'

In [ ]:
ano = 2021
base = '/content/drive/MyDrive/Projeto - RAIS/base/'
saida = '/content/drive/MyDrive/Projeto - RAIS/extract/'

processador = ProcessadorDeDadosRais(proporcao_amostra = 0.01, semente_aleatoria = 42)
processador.processar_dados_e_salvar(ano, 0, base, saida, colunas_a_excluir, baixa_memoria = False)

Acessando o diretório: RAIS_VINC_PUB_CENTRO_OESTE.7z 

Extraindo arquivo: RAIS_VINC_PUB_CENTRO_OESTE.txt
O arquivo já foi extraído anteriormente: /content/drive/MyDrive/Projeto - RAIS/extract/2021/RAIS_VINC_PUB_CENTRO_OESTE.txt

Criando DataFrame para: CENTRO_OESTE


'O dataframe em formato parquet já foi salvo anteriormente.'